# Exponential Smoothing

**Exponential Smoothing** is a time series forecasting method for univariate data that can be extended to support data with a systematic trend or seasonal component. It is widely used in business and economic forecasting.

## Concept

Exponential Smoothing methods are weighted averages of past observations, but the weights decrease exponentially over time. The most recent observations are given relatively more weight in forecasting than the older observations.

## Simple Exponential Smoothing

The simplest form of exponential smoothing is given by the formulas:
$$
\hat{y}_{t+1} = \alpha y_t + (1 - \alpha) \hat{y}_t
$$
where:
- $\hat{y}_{t+1}$ is the forecast for the next period,
- $y_t$ is the actual value at time t,
- $\hat{y}_t$ is the forecast for period t,
- $\alpha$ is the smoothing parameter (0 < $\alpha$ < 1).

## Double Exponential Smoothing

Used when the data show a trend. This method consists of two equations:
1. **Level equation**:
   $$
   l_t = \alpha y_t + (1 - \alpha)(l_{t-1} + b_{t-1})
   $$
2. **Trend equation**:
   $$
   b_t = \beta^*(l_t - l_{t-1}) + (1 - \beta^*)b_{t-1}
   $$
where:
- $l_t$ is the level component,
- $b_t$ is the trend component of the model,
- $\beta^*$ is the smoothing parameter for the trend.

## Triple Exponential Smoothing (Holt-Winters)

This method extends exponential smoothing to capture seasonality. It includes three equations:
1. **Level equation**,
2. **Trend equation**,
3. **Seasonal equation**:
   $$
   s_t = \gamma (y_t - l_{t-1} - b_{t-1}) + (1 - \gamma) s_{t-m}
   $$
where:
- $s_t$ is the seasonal component,
- $\gamma$ is the smoothing parameter for the seasonal component,
- $m$ is the number of periods in a season.

## Applications

Exponential Smoothing is used in various fields including:
- Retail for inventory control,
- Economics for tracking gross domestic, product and other economic indicators,
- Finance for stock price analysis,
- Supply chain management for demand forecasting.

# Implemetation

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import ipywidgets as widgets
from IPython.display import display, clear_output

from sklearn.metrics import mean_squared_error
from IPython.display import display, clear_output, HTML
from statsmodels.tsa.holtwinters import ExponentialSmoothing

import warnings
warnings.filterwarnings("ignore")

### Import and show Data

In [2]:
data = pd.read_csv('./Data/BeadArea.csv')
display(data.head())
print ("The data is composed of ", data.shape[0], " rows and ", data.shape[1], " columns.")

,Time,Position Feedback,Velocity Feedback from Axis 1,Velocity Feedback from Axis 2,Temperature,Current Feedback,Interpolated Bead Area
0,0.056,15.567,11.178,6.4952,1.1173,1.2309,1.1184
1,0.057,15.579,11.257,6.4952,1.1175,1.2735,1.1191
2,0.058,15.590,11.375,3.2476,1.1175,1.1567,1.1198
3,0.059,15.601,11.324,6.4952,1.1173,1.2975,1.1205
4,0.060,15.613,11.285,6.4952,1.1173,1.2168,1.1212


The data is composed of  295568  rows and  7  columns.


### Predict Bead Area

In [3]:

# Define widgets with adjusted layout
index_range_slider = widgets.IntRangeSlider(
    value=[0, min(500, len(data))],
    min=0,
    max=len(data),
    step=1,
    description='Index Range:',
    layout=widgets.Layout(width='600px'),  # Increase width for better readability
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

train_size_slider = widgets.IntSlider(
    value=80,
    min=50,
    max=95,
    step=1,
    description='Train %:',
    layout=widgets.Layout(width='600px'),  # Increase width
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

# Exponential Smoothing parameter sliders
alpha_slider = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.01,
    description='Smoothing Level (α):',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'},
    continuous_update=False
)

beta_slider = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.01,
    description='Smoothing Slope (β):',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'},
    continuous_update=False
)

gamma_slider = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.01,
    description='Smoothing Seasonal (γ):',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '170px'},
    continuous_update=False
)

# Slider to control future prediction steps
future_steps_slider = widgets.IntSlider(
    value=10,
    min=0,
    max=100,
    step=1,
    description='Future Steps:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'},
    continuous_update=False
)

apply_button = widgets.Button(description="Apply Changes", layout=widgets.Layout(width='800px'))

# Define the function to apply changes and update the plots
def apply_changes(b):
    with output:
        clear_output(wait=True)
        
        # Extract the parameters from widgets
        index_range = index_range_slider.value
        train_size_pct = train_size_slider.value / 100
        alpha = alpha_slider.value
        beta = beta_slider.value
        gamma = gamma_slider.value
        future_steps = future_steps_slider.value
        
        # Slice the data
        df = data[index_range[0]:index_range[1]]
        
        # Prepare the data
        y = df['Interpolated Bead Area']
        
        # Train-test split
        train_size = int(len(df) * train_size_pct)
        y_train, y_test = y[:train_size], y[train_size:train_size+future_steps]
        
        # Fit Exponential Smoothing model on training data
        model = ExponentialSmoothing(
            y_train,
            trend='add',  # You can also try 'mul' for multiplicative trend
            seasonal=None,  # Set to 'add' or 'mul' if you have seasonal data
            seasonal_periods=None  # Set to the length of your seasonality, if applicable
        )
        model_fit = model.fit(smoothing_level=alpha, smoothing_slope=beta, smoothing_seasonal=gamma)

        # Forecast on the test data and additional future steps
        forecast_steps = future_steps
        forecast = model_fit.forecast(steps=forecast_steps)
        
        # Extract predictions for test data
        y_pred = forecast[:future_steps]
        #future_predictions = forecast[len(y_test):]
        
        mse = mean_squared_error(y_test, y_pred)
        display(HTML(f'<b>Mean Squared Error: {mse:.5f}</b>'))  # Display MSE in bold
        
        # Plot predicted vs actual
        plt.figure(figsize=(10, 6))
        plt.plot(y_train.index, y_train, label='Training', color='green')
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.axvline(x=y_test.index[-1], color='gray', linestyle='--')  # Mark the end of actual test data
        #future_index = range(y_test.index[-1] + 1, y_test.index[-1] + 1 + len(future_predictions))
        #plt.plot(future_index, future_predictions, label='Future Predictions', color='green', linestyle='--')
        plt.xlabel('Time')
        plt.ylabel('Interpolated Bead Area')
        plt.title('Actual vs Predicted Interpolated Bead Area')
        plt.legend()
        plt.show()
        
        # Calculate loss for each point
        pointwise_mse_loss = (y_test.values - y_pred) ** 2  # Match dimension with y_test
        
        # Plot the pointwise loss
        plt.figure(figsize=(10, 6))
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.plot(y_test.index, pointwise_mse_loss, label='Pointwise MSE Loss', color='orange')
        plt.xlabel('Time')
        plt.ylabel('MSE Loss')
        plt.title('Pointwise MSE Loss of Predicted vs Actual Interpolated Bead Area')
        plt.legend()
        plt.show()

# Link the apply button to the function
apply_button.on_click(apply_changes)

# Display the widgets and the output area
output = widgets.Output()

display(index_range_slider, train_size_slider, alpha_slider, beta_slider, gamma_slider, future_steps_slider, apply_button, output)


IntRangeSlider(value=(0, 500), continuous_update=False, description='Index Range:', layout=Layout(width='600px…

IntSlider(value=80, continuous_update=False, description='Train %:', layout=Layout(width='600px'), max=95, min…

FloatSlider(value=0.5, continuous_update=False, description='Smoothing Level (α):', layout=Layout(width='600px…

FloatSlider(value=0.5, continuous_update=False, description='Smoothing Slope (β):', layout=Layout(width='600px…

FloatSlider(value=0.5, continuous_update=False, description='Smoothing Seasonal (γ):', layout=Layout(width='60…

IntSlider(value=10, continuous_update=False, description='Future Steps:', layout=Layout(width='600px'), style=…

Button(description='Apply Changes', layout=Layout(width='800px'), style=ButtonStyle())

Output()